In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pickle

# 데이터 로드
customer_data = pd.read_csv('DS_Customer_data_modified2.csv')

# 필요한 피처 선택 및 전처리
features = customer_data[['성별', '연령', '결혼여부', '부양자수', '추천횟수', '과금방식', '보안서비스', '백업서비스', '데이터사용량']]
target = customer_data['이탈가능점수']  # 이탈 여부를 예측하는 타겟 변수

# 범주형 데이터 인코딩 (get_dummies로 원-핫 인코딩)
features_encoded = pd.get_dummies(features)

# 데이터 분할 (학습용/테스트용)
X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# 랜덤포레스트 모델 학습
model = RandomForestClassifier()
model.fit(X_train, y_train)

# 모델 저장
with open('churn_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [4]:
import streamlit as st
import pandas as pd
import pickle

# 학습된 모델 불러오기
with open('churn_model.pkl', 'rb') as f:
    model = pickle.load(f)

st.title("고객 이탈 예측 대시보드")

# 상단 요약 카드 스타일 요약 정보
st.metric(label="총 고객 수", value="1,269")
st.metric(label="평균 이탈 가능 점수", value="55%")

# 신규 고객 데이터 입력 섹션
st.header("신규 고객 데이터 입력")

성별 = st.selectbox('성별', ['남성', '여성'])
연령 = st.number_input('연령', min_value=18, max_value=100)
결혼여부 = st.selectbox('결혼 여부', ['Yes', 'No'])
부양자수 = st.number_input('부양자 수', min_value=0)
추천횟수 = st.number_input('추천 횟수', min_value=0)
과금방식 = st.selectbox('과금 방식', ['신용카드', '계좌이체'])
보안서비스 = st.selectbox('보안 서비스 사용 여부', ['Yes', 'No'])
백업서비스 = st.selectbox('백업 서비스 사용 여부', ['Yes', 'No'])
데이터사용량 = st.number_input('데이터 사용량 (GB)', min_value=0)

# 입력된 값을 바탕으로 새로운 데이터를 생성
new_data = {
    '연령': 연령,
    '부양자수': 부양자수,
    '추천횟수': 추천횟수,
    '데이터사용량': 데이터사용량,
    f'성별_{성별}': 1,
    f'결혼여부_{결혼여부}': 1,
    f'과금방식_{과금방식}': 1,
    f'보안서비스_{보안서비스}': 1,
    f'백업서비스_{백업서비스}': 1,
}

# 나머지 값은 0으로 설정 (원-핫 인코딩의 특성상 다른 값은 모두 0이어야 함)
for col in ['성별_남성', '성별_여성', '결혼여부_No', '결혼여부_Yes',
            '과금방식_신용카드', '과금방식_계좌이체',
            '보안서비스_No', '보안서비스_Yes',
            '백업서비스_No', '백업서비스_Yes']:
    if col not in new_data:
        new_data[col] = 0

# DataFrame으로 변환
new_data_df = pd.DataFrame([new_data])

# 예측 버튼 클릭 시 예측 수행
if st.button("이탈 확률 예측"):
    churn_prob = model.predict_proba(new_data_df)[0][1] * 100
    st.write(f"예측된 고객 이탈 확률: {churn_prob:.2f}%")

    # 경고 메시지 출력
    if churn_prob > 50:
        st.warning("고객 이탈 위험이 높습니다!")
    else:
        st.success("고객 이탈 위험이 낮습니다.")


2024-11-14 00:57:55.618 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 00:57:55.618 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 00:57:55.619 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 00:57:55.620 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 00:57:55.621 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 00:57:55.622 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 00:57:55.623 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 00:57:55.624 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

ValueError: X has 5 features, but RandomForestClassifier is expecting 20 features as input.